In [57]:
%pip install langchain llama_index
%pip install --upgrade llama_index langchain



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [58]:
import os

os.environ["OPENAI_API_KEY"] = os.environ['API_KEY']


In [93]:
import requests

url = 'https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/001916/files/61171_74826.html'
res = requests.get(url)
res.encoding = res.apparent_encoding
text = res.text

print(text)



<?xml version="1.0" encoding="Shift_JIS"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN"
    "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ja" >
<head>
	<meta http-equiv="Content-Type" content="text/html;charset=Shift_JIS" />
	<meta http-equiv="content-style-type" content="text/css" />
	<link rel="stylesheet" type="text/css" href="../../aozora.css" />
	<title>円城塔 鉄道模型の夜</title>
	<script type="text/javascript" src="../../jquery-1.4.2.min.js"></script>
  <link rel="Schema.DC" href="http://purl.org/dc/elements/1.1/" />
	<meta name="DC.Title" content="鉄道模型の夜" />
	<meta name="DC.Creator" content="円城塔" />
	<meta name="DC.Publisher" content="青空文庫" />
</head>
<body>
<div class="metadata">
<h1 class="title">鉄道模型の夜</h1>
<h2 class="author">円城塔</h2>
<br />
<br />
</div>
<div id="contents" style="display:none"></div><div class="main_text"><br />
　彼の趣味は箱庭であり、鉄道の方はあとからついてきた。だから最初は、小さな庭でも家のミニチュアでも構わなかった。かといって、押絵を風呂敷に包んで旅するような趣味があるわけでなし、縮小するのは持

In [94]:
with open('./data/61171_74826.html', 'w', encoding='utf-8') as f:
     f.write(text)

In [61]:
from llama_index import download_loader, GPTSimpleVectorIndex
from pathlib import Path

In [62]:
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)

In [63]:
loader = UnstructuredReader()
documents = loader.load_data(file=Path(f'./data/61171_74826.html'), 
                            #  split_documents=False
                            )


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/takeshiiijima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/takeshiiijima/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...


In [64]:
documents


[Document(text='', doc_id='f107b6b0-14f8-4892-b662-4df91335796a', embedding=None, doc_hash='dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758e8014a24c7fa039e91', extra_info=None)]

In [65]:
import os
os.environ['OPENAI_API_KEY'] =  os.environ['API_KEY']


In [66]:
from gpt_index import GPTTreeIndex, SimpleDirectoryReader, LLMPredictor, GPTSimpleVectorIndex, GPTListIndex, Prompt, ServiceContext
from gpt_index.indices.base import BaseGPTIndex
from gpt_index.langchain_helpers.text_splitter import TokenTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from gpt_index.response.schema import Response
import pandas as pd
from typing import Tuple



Setup benchmark

In [67]:
from dataclasses import dataclass
from typing import List

In [68]:
@dataclass
class TestCase:
    query: str 
    must_contain: List[str]

In [69]:
@dataclass
class TestOutcome:
    test: TestCase
    response: Response
    
    @property
    def is_correct_response(self) -> bool:
        is_correct = True
        for answer in self.test.must_contain:
            if answer not in self.response.response:
                is_correct = False
        return is_correct
    
    @property
    def is_correct_source(self) -> bool:
        is_correct = True
        for answer in self.test.must_contain:
            if all(answer not in node.source_text for node in self.response.source_nodes):
                is_correct = False
        return is_correct

In [70]:
class Benchmark:
    def __init__(self, tests: List[TestCase]) -> None:
        self._tests = tests
    
    def test(self, index: BaseGPTIndex, llm_predictor: LLMPredictor, **kwargs) -> List[TestOutcome]:
        outcomes: List[TestOutcome] = []
        service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
        for test in self._tests:
            response = index.query(
                test.query,
                service_context=service_context,
                **kwargs
            )
            outcome = TestOutcome(test=test, response=response)
            outcomes.append(outcome)
        return outcomes

In [71]:
def analyze_outcome(outcomes: List[TestOutcome]) -> None:
    rows = []
    for outcome in outcomes:
        row = [outcome.test.query, outcome.is_correct_response, outcome.is_correct_source]
        rows.append(row)
    df = pd.DataFrame(rows, columns=['Test Query', 'Correct Response', 'Correct Source'])
    return df


In [72]:
test_shumi = TestCase(
    query="彼の趣味はなんですか？",
    must_contain=["箱庭"]
)


In [83]:
bm = Benchmark([test_shumi])


LLM based evaluation

In [84]:
from gpt_index.prompts.prompt_type import PromptType

EVAL_PROMPT_TMPL = (
    "Given the question below. \n"
    "---------------------\n"
    "{query_str}"
    "\n---------------------\n"
    "Decide if the following retreived context is relevant. \n"
    "\n---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Then decide if the answer is correct. \n"
    "\n---------------------\n"
    "{answer_str}"
    "\n---------------------\n"
    "Answer in the following format:\n"
    "'Context is relevant: <True>\nAnswer is correct: <True>' "
    "and explain why."
)

class EvalPrompt(Prompt):
    prompt_type: PromptType = PromptType.CUSTOM
    input_variables: List[str] = ["query_str", 'context_str', 'answer_str']

DEFAULT_EVAL_PROMPT = EvalPrompt(EVAL_PROMPT_TMPL)



In [85]:
import re

def extract_eval_result(result_str: str):
    boolean_pattern = r"(True|False)"
    matches = re.findall(boolean_pattern, result_str)
    return [match == "True" for match in matches]  

In [86]:
def analyze_outcome_llm_single(outcome: TestOutcome, llm_predictor: LLMPredictor) -> Tuple[bool, bool]:
    try:
        source_text = outcome.response.source_nodes[0].source_text
    except:
        source_text = "Failed to retrieve any context"
        
    result_str, _ = llm_predictor.predict(
        DEFAULT_EVAL_PROMPT,
        query_str=outcome.test.query,
        context_str=source_text,
        answer_str=outcome.response.response
    )
 
    is_context_relevant, is_answer_correct = extract_eval_result(result_str)
    return is_answer_correct, is_context_relevant, result_str

In [87]:
def analyze_outcome_llm(outcomes: List[TestOutcome], llm_predictor: LLMPredictor) -> None:
    rows = []
    for outcome in outcomes:
        is_correct_response, is_correct_source, result_str = analyze_outcome_llm_single(outcome, llm_predictor)
        row = [outcome.test.query, is_correct_response, is_correct_source, result_str]
        rows.append(row)
    df = pd.DataFrame(rows, columns=['Test Query', 'Correct Response (LLM)', 'Correct Source (LLM)', 'Eval (LLM)'])
    return df



Build Indices

In [88]:
vector_index = GPTSimpleVectorIndex.from_documents(documents)
list_index = GPTListIndex.from_documents(documents)
tree_index = GPTTreeIndex.from_documents(documents)



INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [89]:
vector_index.save_to_disk('vector_index.json')
tree_index.save_to_disk('tree_index.json')
list_index.save_to_disk('list_index.json')



In [90]:
# Load indices
tree_index = GPTTreeIndex.load_from_disk('tree_index.json')
list_index = GPTListIndex.load_from_disk('list_index.json')
vector_index = GPTSimpleVectorIndex.load_from_disk('vector_index.json')



Create LLMPredictors

In [91]:
# gpt-4
llm_predictor_gpt4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
# gpt-3 (text-davinci-003)
llm_predictor_gpt3 = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003"))
# chatgpt (gpt-3.5-turbo)
llm_predictor_chatgpt = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))


Benchmarking

In [92]:
outcomes_tree_gpt4 = bm.test(tree_index, llm_predictor_gpt4)


INFO:gpt_index.indices.tree.leaf_query:> Starting query: 彼の趣味はなんですか？


ZeroDivisionError: integer division or modulo by zero

In [ ]:
analyze_outcome(outcomes_tree_gpt4)


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
outcomes_tree_gpt3 = bm.test(tree_index, llm_predictor_gpt3)


INFO:gpt_index.indices.tree.leaf_query:> Starting query: 彼の趣味はなんですか？


ZeroDivisionError: integer division or modulo by zero

In [ ]:
analyze_outcome(outcomes_tree_gpt3)


NameError: name 'outcomes_tree_gpt3' is not defined

In [ ]:
outcomes_vector_gpt4 = bm.test(vector_index, llm_predictor_gpt4)

INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 17 tokens


In [ ]:
analyze_outcome(outcomes_vector_gpt4)

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
outcomes_vector_gpt3 = bm.test(vector_index, llm_predictor_gpt3)

INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 17 tokens


In [ ]:
analyze_outcome(outcomes_vector_gpt3)

TypeError: argument of type 'NoneType' is not iterable

LLM based Evaluation

In [ ]:
analyze_outcome(outcomes_vector_gpt3)

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
eval_gpt4 = analyze_outcome_llm(outcomes_vector_gpt3, llm_predictor_gpt4)

In [ ]:
eval_gpt4

,Test Query,Correct Response (LLM),Correct Source (LLM),Eval (LLM)
0,彼の趣味はなんですか？,True,True,Context is relevant: True\nAnswer is correct: ...


In [ ]:
eval_chatgpt = analyze_outcome_llm(outcomes_vector_gpt3, llm_predictor_chatgpt)

/Users/takeshiiijima/opt/anaconda3/lib/python3.9/site-packages/gpt_index/data_structs/node_v2.py:144: UserWarning: .source_text is deprecated, use .node.get_text() instead
  warnings.warn(".source_text is deprecated, use .node.get_text() instead")


In [ ]:
eval_chatgpt

,Test Query,Correct Response (LLM),Correct Source (LLM),Eval (LLM)
0,彼の趣味はなんですか？,True,True,Context is relevant: True\nAnswer is correct: ...
